In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import warnings
import glob
from difflib import SequenceMatcher
import os
import re
import string
from difflib import get_close_matches
import os
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Shows plots in jupyter notebook
%matplotlib inline
import sys
import time

# Set plot style
sns.set(color_codes=True)

In [3]:
import os  # Ensure os is imported

csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
csv_files.sort()  # Sort alphabetically

print(f"Found {len(csv_files)} CSV files in the current directory.\n")

# Create a summary of all files
print("CSV Files Summary:")
print(f"{'File Name':<25} {'Rows':<10} {'Columns':<10} {'Size (KB)':<15}")
print("-" * 60)

for file in csv_files:
    try:
        file_size = os.path.getsize(file) / 1024
        df = pd.read_csv(file)
        print(f"{file:<25} {df.shape[0]:<10} {df.shape[1]:<10} {file_size:.2f} KB")
        
    except Exception as e:
        print(f"{file:<25} Error: {str(e)}")

print("\nDetailed information for each file:")
print("=" * 80)

# Read each CSV file and display basic info
for file in csv_files:
    try:
        print(f"\nFile: {file}")
        print("-" * 80)
        
        # Read the file
        df = pd.read_csv(file)
        
        # Display basic info
        print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
        print("\nColumns:")
        for col in df.columns:
            print(f"  - {col}")
        
        # Display first 3 rows
        print("\nFirst 3 rows:")
        print(df.head(3))
        
        print("-" * 80)
        
    except Exception as e:
        print(f"Error reading {file}: {str(e)}")
        print("-" * 80)

Found 0 CSV files in the current directory.

CSV Files Summary:
File Name                 Rows       Columns    Size (KB)      
------------------------------------------------------------

Detailed information for each file:


In [4]:
import os


csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]

# Print info for each CSV file
for file in csv_files:
    print(f"\n{'='*80}")
    print(f"FILE: {file}")
    print(f"{'='*80}")
    try:
        df = pd.read_csv(file)
        print(f"Shape: {df.shape}")
        print("\nInfo:")
        df.info()
        print("\nSample (first 3 rows):")
        print(df.head(3))
    except Exception as e:
        print(f"Error reading file: {e}")
        

In [5]:
# Get all CSV files in the current directory
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]

# Print describe for each CSV file
for file in csv_files:
    print(f"\n{'='*80}")
    print(f"FILE: {file}")
    print(f"{'='*80}")
    try:
        df = pd.read_csv(file)
        print(f"Shape: {df.shape}")
        
        # Get numeric columns
        numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
        
        if numeric_cols:
            print("\nNumeric Columns Statistics:")
            print(df[numeric_cols].describe())
        else:
            print("\nNo numeric columns to describe.")
            
        # Get categorical columns (limit to those with fewer than 20 unique values)
        cat_cols = [col for col in df.columns if col not in numeric_cols 
                   and df[col].nunique() < 20 and df[col].nunique() > 0]
        
        if cat_cols:
            print("\nCategorical Columns (Top 5 value counts):")
            for col in cat_cols[:5]:  # Limit to first 5 categorical columns
                print(f"\n{col}:")
                print(df[col].value_counts().head())
        else:
            print("\nNo suitable categorical columns to describe.")
            
    except Exception as e:
        print(f"Error processing file: {e}")

In [6]:



def load_csv_files():
    """Load all CSV files in the current directory into a dictionary of DataFrames."""
    # Get all CSV files in the current directory
    csv_files = glob.glob('*.csv')
    
    # Create a dictionary to store all dataframes
    dfs = {}
    
    # Read each CSV file
    for file in csv_files:
        # Get the name without extension
        name = file.split('.')[0]
        # Read the CSV file
        dfs[name] = pd.read_csv(file)
        print(f'Read {file} successfully')
    
    return dfs

def create_entity_resolution_table(dfs):
    """Create an entity resolution table for diseases across all files."""
    # Get unique diseases from each file
    diseases = {}
    for name, df in dfs.items():
        if 'Disease' in df.columns:
            diseases[name] = set(df['Disease'].unique())
    
    # Get all unique diseases across all files
    all_diseases = set()
    for disease_set in diseases.values():
        all_diseases.update(disease_set)
    
    # Create a DataFrame to track which diseases appear in which files
    entity_df = pd.DataFrame(index=sorted(all_diseases))
    
    # Fill the DataFrame with True/False values indicating presence in each file
    for name, disease_set in diseases.items():
        entity_df[name] = entity_df.index.isin(disease_set)
    
    # Add a column for potential issues (diseases not in all files)
    entity_df['in_all_files'] = entity_df.all(axis=1)
    
    return entity_df

def find_similar_diseases(diseases_list, threshold=0.8):
    """Find similar disease names using string similarity."""
    similar_pairs = []
    diseases_list = sorted(diseases_list)
    
    for i, disease1 in enumerate(diseases_list):
        for disease2 in diseases_list[i+1:]:
            similarity = SequenceMatcher(None, disease1.lower(), disease2.lower()).ratio()
            if similarity > threshold and disease1 != disease2:
                similar_pairs.append((disease1, disease2, similarity))
    
    return sorted(similar_pairs, key=lambda x: x[2], reverse=True)

def apply_disease_mapping(df, mapping_dict):
    """Apply disease name mapping to a DataFrame."""
    if 'Disease' in df.columns:
        return df.replace({'Disease': mapping_dict})
    return df

def create_comprehensive_table(dfs, entity_df):
    """Create a comprehensive table linking diseases with symptoms, descriptions, and precautions."""
    # Get all unique diseases from the entity resolution table
    unique_diseases = entity_df.index.tolist()
    
    # Create a new DataFrame for the comprehensive table
    comprehensive_df = pd.DataFrame(index=unique_diseases)
    
    # Add disease descriptions if available
    if 'symptom_Description' in dfs and 'Disease' in dfs['symptom_Description'].columns:
        # Create a dictionary mapping diseases to descriptions
        descriptions = dict(zip(dfs['symptom_Description']['Disease'], dfs['symptom_Description']['Description']))
        comprehensive_df['Description'] = comprehensive_df.index.map(descriptions)
    
    # Add precautions if available
    if 'symptom_precaution' in dfs and 'Disease' in dfs['symptom_precaution'].columns:
        # Create dictionaries for each precaution column
        for i in range(1, 5):
            col_name = f'Precaution_{i}'
            if col_name in dfs['symptom_precaution'].columns:
                precautions = dict(zip(dfs['symptom_precaution']['Disease'], dfs['symptom_precaution'][col_name]))
                comprehensive_df[col_name] = comprehensive_df.index.map(precautions)
    
    # Add symptoms if available (from dataset.csv)
    if 'dataset' in dfs and 'Disease' in dfs['dataset'].columns:
        # Get all symptom columns
        symptom_cols = [col for col in dfs['dataset'].columns if 'Symptom_' in col]
        
        # Create a dictionary mapping diseases to their symptoms
        disease_symptoms = {}
        for _, row in dfs['dataset'].iterrows():
            disease = row['Disease']
            symptoms = [row[col] for col in symptom_cols if pd.notna(row[col]) and row[col] != '']
            if disease in disease_symptoms:
                disease_symptoms[disease].update(symptoms)
            else:
                disease_symptoms[disease] = set(symptoms)
        
        # Convert sets to sorted lists
        for disease in disease_symptoms:
            disease_symptoms[disease] = sorted(list(disease_symptoms[disease]))
        
        # Add to comprehensive table
        comprehensive_df['Symptoms'] = comprehensive_df.index.map(disease_symptoms)
    
    return comprehensive_df

def standardize_disease_names(dfs):
    """Standardize disease names across all files using a predefined mapping."""
    # Define name mappings
    disease_name_mapping = {
        "Dimorphic hemmorhoids(piles)": "Dimorphic Hemorrhoids (Piles)",
        "Dimorphic hemorrhoids(piles)": "Dimorphic Hemorrhoids (Piles)",
        "Osteoarthristis": "Osteoarthritis",
        "Peptic ulcer diseae": "Peptic Ulcer Disease",
        "(vertigo) Paroymsal  Positional Vertigo": "Paroxysmal Positional Vertigo",
        "Diabetes ": "Diabetes"
    }
    
    # Apply mapping to all dataframes
    standardized_dfs = {}
    for name, df in dfs.items():
        if 'Disease' in df.columns:
            standardized_dfs[name] = apply_disease_mapping(df, disease_name_mapping)
            print(f'Applied disease name mapping to {name}')
        else:
            standardized_dfs[name] = df
    
    return standardized_dfs

def save_standardized_files(dfs):
    """Save standardized DataFrames to CSV files."""
    # Create a directory for standardized files if it doesn't exist
    if not os.path.exists('standardized'):
        os.makedirs('standardized')
    
    # Save each DataFrame to a CSV file
    for name, df in dfs.items():
        file_path = os.path.join('standardized', f'{name}.csv')
        df.to_csv(file_path, index=False)
        print(f'Saved standardized file to {file_path}')

def save_comprehensive_table(comprehensive_df):
    """Save the comprehensive table to a CSV file."""
    # Create a directory for output files if it doesn't exist
    if not os.path.exists('output'):
        os.makedirs('output')
    
    # Save the comprehensive table
    file_path = os.path.join('output', 'comprehensive_disease_data.csv')
    comprehensive_df.to_csv(file_path)
    print(f'Saved comprehensive table to {file_path}')

def main():
    """Main function to run the entity resolution process."""
    print("Loading CSV files...")
    dfs = load_csv_files()
    
    print("\nCreating entity resolution table...")
    entity_df = create_entity_resolution_table(dfs)
    
    # Display the entity resolution table
    print("\nEntity Resolution Table:")
    print(entity_df)
    
    # Show diseases that are not in all files
    missing_diseases = entity_df[~entity_df['in_all_files']]
    if len(missing_diseases) > 0:
        print("\nDiseases not in all files (potential entity resolution issues):")
        print(missing_diseases)
    else:
        print("\nAll diseases appear in all files.")
    
    # Find similar disease names
    all_diseases = entity_df.index.tolist()
    similar_diseases = find_similar_diseases(all_diseases)
    
    if similar_diseases:
        print("\nPotential similar disease names that might need standardization:")
        similar_df = pd.DataFrame(similar_diseases, columns=['Disease 1', 'Disease 2', 'Similarity'])
        print(similar_df)
    else:
        print("\nNo similar disease names found.")
    
    # Standardize disease names
    print("\nStandardizing disease names...")
    standardized_dfs = standardize_disease_names(dfs)
    
    # Create comprehensive table
    print("\nCreating comprehensive table...")
    comprehensive_df = create_comprehensive_table(standardized_dfs, entity_df)
    
    # Save standardized files
    print("\nSaving standardized files...")
    save_standardized_files(standardized_dfs)
    
    # Save comprehensive table
    print("\nSaving comprehensive table...")
    save_comprehensive_table(comprehensive_df)
    
    print("\nEntity resolution process completed successfully!")

if __name__ == "__main__":
    main()


Loading CSV files...

Creating entity resolution table...

Entity Resolution Table:
Empty DataFrame
Columns: [in_all_files]
Index: []

All diseases appear in all files.

No similar disease names found.

Standardizing disease names...

Creating comprehensive table...

Saving standardized files...

Saving comprehensive table...
Saved comprehensive table to output\comprehensive_disease_data.csv

Entity resolution process completed successfully!


In [7]:


def load_csv_files():
    """Load all CSV files in the current directory into a dictionary of DataFrames."""
    # Get all CSV files in the current directory
    csv_files = glob.glob('*.csv')
    
    # Create a dictionary to store all dataframes
    dfs = {}
    
    # Read each CSV file
    for file in csv_files:
        # Get the name without extension
        name = file.split('.')[0]
        # Read the CSV file
        dfs[name] = pd.read_csv(file)
        print(f'Read {file} successfully')
    
    return dfs

def check_missing_values(dfs):
    """Check for missing values in all DataFrames."""
    for name, df in dfs.items():
        print(f'\n{name} - Missing Values:')
        missing = df.isnull().sum()
        if missing.sum() > 0:
            print(missing[missing > 0])
        else:
            print('No missing values')

def check_duplicates(dfs):
    """Check for duplicate rows in all DataFrames."""
    for name, df in dfs.items():
        duplicates = df.duplicated().sum()
        print(f'\n{name} - Duplicates:')
        if duplicates > 0:
            print(f'Found {duplicates} duplicate rows')
        else:
            print('No duplicate rows found')

def clean_dataframe(df):
    """Apply comprehensive cleaning to a DataFrame."""
    # Make a copy to avoid modifying the original
    df_clean = df.copy()
    
    print(f'Missing values before cleaning:\n{df_clean.isnull().sum()}')
    
    # For numeric columns, fill with median
    numeric_cols = df_clean.select_dtypes(include=['number']).columns
    for col in numeric_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    
    # For categorical columns, fill with mode
    cat_cols = df_clean.select_dtypes(include=['object']).columns
    for col in cat_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])
    
    print(f'\nMissing values after cleaning:\n{df_clean.isnull().sum()}')
    
    # Remove duplicates
    duplicates = df_clean.duplicated().sum()
    if duplicates > 0:
        print(f'\nFound {duplicates} duplicate rows')
        df_clean = df_clean.drop_duplicates()
        print(f'Removed all duplicate rows')
    else:
        print('\nNo duplicate rows found')
    
    # Standardize text data (lowercase for text columns)
    for col in cat_cols:
        if df_clean[col].dtype == 'object':
            df_clean[col] = df_clean[col].str.strip().str.lower()
    
    return df_clean

def clean_all_dataframes(dfs):
    """Apply cleaning to all DataFrames."""
    cleaned_dfs = {}
    for name, df in dfs.items():
        print(f'\nCleaning {name}...')
        cleaned_dfs[name] = clean_dataframe(df)
    return cleaned_dfs

def save_cleaned_files(dfs):
    """Save cleaned DataFrames to CSV files."""
    # Create a directory for cleaned files if it doesn't exist
    if not os.path.exists('cleaned'):
        os.makedirs('cleaned')
    
    # Save each DataFrame to a CSV file
    for name, df in dfs.items():
        file_path = os.path.join('cleaned', f'{name}.csv')
        df.to_csv(file_path, index=False)
        print(f'Saved cleaned file to {file_path}')

def main():
    """Main function to run the data cleaning process."""
    print("Loading CSV files...")
    dfs = load_csv_files()
    
    print("\nChecking for missing values...")
    check_missing_values(dfs)
    
    print("\nChecking for duplicates...")
    check_duplicates(dfs)
    
    # Ask user if they want to clean all files
    clean_all = input("\nDo you want to clean all files? (y/n): ")
    if clean_all.lower() == 'y':
        print("\nCleaning all files...")
        cleaned_dfs = clean_all_dataframes(dfs)
        
        # Ask user if they want to save cleaned files
        save_files = input("\nDo you want to save cleaned files? (y/n): ")
        if save_files.lower() == 'y':
            print("\nSaving cleaned files...")
            save_cleaned_files(cleaned_dfs)
    
    print("\nData cleaning process completed!")

if __name__ == "__main__":
    main()


Loading CSV files...

Checking for missing values...

Checking for duplicates...

Data cleaning process completed!


In [9]:


def print_header(title):
    """Print a formatted header."""
    print("\n" + "=" * 80)
    print(f"{title.center(80)}")
    print("=" * 80 + "\n")

def main():
    """Main function to run the health data processing pipeline."""
    print_header("HEALTH DATA PROCESSOR")
    
    print("This script provides options to clean, analyze, and resolve entities in health data CSV files.")
    print("Choose an option from the menu below:")
    
    while True:
        print("\nMENU:")
        print("1. Clean Data")
        print("2. Analyze Data")
        print("3. Entity Resolution")
        print("4. Run All")
        print("5. Exit")
        
        choice = input("\nEnter your choice (1-5): ")
        
        if choice == '1':
            print_header("DATA CLEANING")
            import data_cleaning
            data_cleaning.main()
        
        elif choice == '2':
            print_header("DATA ANALYSIS")
            import data_analysis
            data_analysis.main()
        
        elif choice == '3':
            print_header("ENTITY RESOLUTION")
            import entity_resolution
            entity_resolution.main()
        
        elif choice == '4':
            print_header("RUNNING COMPLETE PIPELINE")
            
            print("Step 1: Data Cleaning")
            import data_cleaning
            data_cleaning.main()
            
            print("\nStep 2: Entity Resolution")
            import entity_resolution
            entity_resolution.main()
            
            print("\nStep 3: Data Analysis")
            import data_analysis
            data_analysis.main()
            
            print("\nComplete pipeline executed successfully!")
        
        elif choice == '5':
            print("\nExiting Health Data Processor. Goodbye!")
            sys.exit(0)
        
        else:
            print("\nInvalid choice. Please enter a number between 1 and 5.")
        
        time.sleep(1)

if __name__ == "__main__":
    main()




                             HEALTH DATA PROCESSOR                              

This script provides options to clean, analyze, and resolve entities in health data CSV files.
Choose an option from the menu below:

MENU:
1. Clean Data
2. Analyze Data
3. Entity Resolution
4. Run All
5. Exit

                                 DATA CLEANING                                  



ModuleNotFoundError: No module named 'data_cleaning'

In [10]:
import pandas as pd
import numpy as np
import string
import random
import os
import sys

class EnhancedDiagnosisSystem:
    """An enhanced diagnosis system with physical parameters and dietary recommendations."""
    
    def __init__(self):
        """Initialize the enhanced diagnosis system."""
        print("Loading comprehensive medical data...")
        
        # Load all CSV files
        self.dataset = pd.read_csv('./Datasets/dataset.csv')
        self.descriptions = pd.read_csv('./Datasets/symptom_Description.csv')
        self.precautions = pd.read_csv('./Datasets/symptom_precaution.csv')
        self.severity = pd.read_csv('./Datasets/symptom_severity.csv')
        
        # Extract all symptoms and create dictionaries
        self.all_symptoms = self.extract_all_symptoms()
        self.disease_symptom_dict = self.create_disease_symptom_dict()
        self.symptom_severity_dict = self.create_symptom_severity_dict()
        
        # Fixed number of questions
        self.num_questions = 8
        
        # Confirmed and denied symptoms
        self.confirmed_symptoms = set()
        self.denied_symptoms = set()
        
        # Physical parameters
        self.physical_params = {}
        
        # Dietary recommendations by disease category
        self.diet_recommendations = {
            'respiratory': [
                "Consume foods rich in vitamin C like citrus fruits and bell peppers",
                "Include anti-inflammatory foods like turmeric and ginger",
                "Stay well-hydrated with water and herbal teas",
                "Avoid dairy products which may increase mucus production",
                "Consume warm broths and soups to ease respiratory symptoms"
            ],
            'digestive': [
                "Eat smaller, more frequent meals",
                "Include easily digestible foods like rice, bananas, and toast",
                "Stay hydrated with clear fluids",
                "Avoid spicy, fatty, and acidic foods",
                "Consider probiotic foods like yogurt to support gut health"
            ],
            'infectious': [
                "Increase protein intake to support immune function",
                "Consume vitamin C rich foods to boost immunity",
                "Stay well-hydrated with water and electrolyte drinks",
                "Include zinc-rich foods like nuts and seeds",
                "Eat light, easily digestible meals"
            ],
            'cardiovascular': [
                "Follow a low-sodium diet",
                "Include heart-healthy fats like olive oil and avocados",
                "Eat plenty of fruits, vegetables, and whole grains",
                "Limit red meat and processed foods",
                "Consider omega-3 rich foods like fatty fish"
            ],
            'neurological': [
                "Include omega-3 fatty acids from fish or flaxseeds",
                "Consume antioxidant-rich berries and dark chocolate",
                "Stay hydrated and limit caffeine",
                "Include vitamin E rich foods like nuts and seeds",
                "Consider magnesium-rich foods like leafy greens"
            ],
            'dermatological': [
                "Increase omega-3 fatty acids for skin health",
                "Include zinc-rich foods like nuts and seeds",
                "Stay well-hydrated to support skin hydration",
                "Consume vitamin A and E rich foods",
                "Limit dairy and sugar which may exacerbate skin conditions"
            ],
            'general': [
                "Maintain a balanced diet with plenty of fruits and vegetables",
                "Stay well-hydrated with water throughout the day",
                "Include lean proteins to support recovery",
                "Consume whole grains for sustained energy",
                "Limit processed foods, sugar, and alcohol"
            ]
        }
        
        # Disease categories
        self.disease_categories = {
            'respiratory': ['Pneumonia', 'Bronchial Asthma', 'Common Cold', 'Tuberculosis'],
            'digestive': ['Gastroenteritis', 'GERD', 'Peptic Ulcer Disease', 'Jaundice'],
            'infectious': ['Malaria', 'Dengue', 'Typhoid', 'Chicken Pox', 'Hepatitis A'],
            'cardiovascular': ['Hypertension', 'Heart Attack', 'Varicose Veins'],
            'neurological': ['Migraine', 'Cervical Spondylosis', 'Paralysis', 'Vertigo'],
            'dermatological': ['Acne', 'Psoriasis', 'Impetigo'],
            'general': ['Diabetes', 'Hypothyroidism', 'Hyperthyroidism', 'Arthritis']
        }
        
        print("Medical data loaded successfully!")
    
    def extract_all_symptoms(self):
        """Extract all unique symptoms from the dataset."""
        symptom_cols = [col for col in self.dataset.columns if 'Symptom_' in col]
        all_symptoms = set()
        for col in symptom_cols:
            symptoms = self.dataset[col].dropna().unique()
            all_symptoms.update([s.strip() for s in symptoms if s.strip()])
        return sorted(list(all_symptoms))
    
    def create_disease_symptom_dict(self):
        """Create a dictionary mapping diseases to their symptoms."""
        disease_symptom_dict = {}
        symptom_cols = [col for col in self.dataset.columns if 'Symptom_' in col]
        
        for disease in self.dataset['Disease'].unique():
            disease_rows = self.dataset[self.dataset['Disease'] == disease]
            symptoms = set()
            for _, row in disease_rows.iterrows():
                for col in symptom_cols:
                    if pd.notna(row[col]) and row[col] != '':
                        symptoms.add(row[col].strip())
            disease_symptom_dict[disease] = sorted(list(symptoms))
        
        return disease_symptom_dict
    
    def create_symptom_severity_dict(self):
        """Create a dictionary mapping symptoms to their severity."""
        severity_dict = {}
        
        # Use symptom_severity.csv if available
        if self.severity is not None:
            for _, row in self.severity.iterrows():
                severity_dict[row['Symptom']] = row['weight']
        
        # Default severity for symptoms not in the files
        for symptom in self.all_symptoms:
            if symptom not in severity_dict:
                severity_dict[symptom] = 1
        
        return severity_dict
    
    def extract_symptoms_from_text(self, text):
        """Extract symptoms from natural language text."""
        text = text.lower().translate(str.maketrans('', '', string.punctuation))
        extracted_symptoms = []
        for symptom in self.all_symptoms:
            clean_symptom = symptom.replace('_', ' ').lower()
            if clean_symptom in text:
                extracted_symptoms.append(symptom)
        return extracted_symptoms
    
    def get_symptom_severity(self, symptom):
        """Get the severity of a symptom."""
        return self.symptom_severity_dict.get(symptom, 1)
    
    def predict_disease(self):
        """Predict diseases based on confirmed symptoms with severity weighting."""
        if not self.confirmed_symptoms:
            return None, 0
        
        # Calculate scores for each disease
        disease_scores = {}
        for disease, disease_symptoms in self.disease_symptom_dict.items():
            matching = self.confirmed_symptoms & set(disease_symptoms)
            denied = self.denied_symptoms & set(disease_symptoms)
            
            if matching:
                # Calculate severity-weighted score
                matching_severity = sum(self.get_symptom_severity(s) for s in matching)
                total_severity = sum(self.get_symptom_severity(s) for s in disease_symptoms)
                
                # Base score is the proportion of severity-weighted symptoms
                score = matching_severity / total_severity if total_severity > 0 else 0
                
                # Penalize for denied symptoms
                if denied:
                    denied_severity = sum(self.get_symptom_severity(s) for s in denied)
                    penalty = denied_severity / total_severity if total_severity > 0 else 0
                    score = max(0, score - penalty)
                
                # Adjust score based on physical parameters if available
                if 'temperature' in self.physical_params:
                    temp = self.physical_params['temperature']
                    # Increase score for fever-related diseases if temperature is high
                    if temp > 38 and any(s in ['fever', 'high_fever'] for s in matching):
                        score *= 1.2
                
                disease_scores[disease] = score
        
        # Find the disease with the highest score
        if disease_scores:
            top_disease = max(disease_scores.items(), key=lambda x: x[1])
            return top_disease[0], top_disease[1]
        
        return None, 0
    
    def get_next_question(self):
        """Get the next best question to ask based on current symptoms."""
        # Make a prediction based on current symptoms
        predicted_disease, _ = self.predict_disease()
        
        if not predicted_disease:
            # If no prediction yet, ask about common symptoms
            common_symptoms = ['fever', 'headache', 'fatigue', 'cough', 'pain', 'nausea', 'vomiting', 'diarrhea']
            for symptom_word in common_symptoms:
                matching_symptoms = [s for s in self.all_symptoms if symptom_word in s.lower()]
                for symptom in matching_symptoms:
                    if symptom not in self.confirmed_symptoms and symptom not in self.denied_symptoms:
                        return symptom
            
            # If all common symptoms asked, pick a random unasked symptom
            unasked = [s for s in self.all_symptoms if s not in self.confirmed_symptoms and s not in self.denied_symptoms]
            if unasked:
                return random.choice(unasked)
            
            return None
        
        # Get symptoms for the predicted disease
        disease_symptoms = set(self.disease_symptom_dict[predicted_disease])
        
        # Find unasked symptoms for this disease
        unasked_symptoms = disease_symptoms - self.confirmed_symptoms - self.denied_symptoms
        
        if unasked_symptoms:
            # Sort by severity
            sorted_symptoms = sorted(unasked_symptoms, key=self.get_symptom_severity, reverse=True)
            return sorted_symptoms[0]
        
        # If all symptoms for the predicted disease have been asked,
        # find symptoms that could differentiate between similar diseases
        
        # Get diseases with some matching symptoms
        candidate_diseases = []
        for disease, symptoms in self.disease_symptom_dict.items():
            if self.confirmed_symptoms & set(symptoms):
                candidate_diseases.append(disease)
        
        # Get all symptoms for candidate diseases
        all_candidate_symptoms = set()
        for disease in candidate_diseases:
            all_candidate_symptoms.update(self.disease_symptom_dict[disease])
        
        # Find unasked symptoms
        unasked_symptoms = all_candidate_symptoms - self.confirmed_symptoms - self.denied_symptoms
        
        if unasked_symptoms:
            # Sort by how many diseases have this symptom (most discriminative first)
            symptom_counts = {}
            for symptom in unasked_symptoms:
                count = sum(1 for disease in candidate_diseases if symptom in self.disease_symptom_dict[disease])
                # Most discriminative are those that appear in about half the diseases
                symptom_counts[symptom] = abs(count - len(candidate_diseases) / 2)
            
            # Get the most discriminative symptom
            sorted_symptoms = sorted(symptom_counts.items(), key=lambda x: x[1])
            return sorted_symptoms[0][0] if sorted_symptoms else None
        
        return None
    
    def format_question(self, symptom, question_num):
        """Format a doctor-like question about a symptom."""
        clean_symptom = symptom.replace('_', ' ').lower()
        
        # Doctor-like question templates
        templates = [
            f"Question {question_num}/8: Have you been experiencing {clean_symptom}?",
            f"Question {question_num}/8: Are you suffering from {clean_symptom}?",
            f"Question {question_num}/8: Do you have any {clean_symptom}?",
            f"Question {question_num}/8: Have you noticed {clean_symptom}?",
            f"Question {question_num}/8: I'd like to know if you have {clean_symptom}?"
        ]
        
        # Add follow-up templates for later questions
        if question_num > 3:
            templates.extend([
                f"Question {question_num}/8: Based on your symptoms, I need to ask: do you have {clean_symptom}?",
                f"Question {question_num}/8: To narrow down the diagnosis, are you experiencing {clean_symptom}?",
                f"Question {question_num}/8: This is important: have you had any {clean_symptom}?"
            ])
        
        return random.choice(templates)
    
    def collect_physical_parameters(self):
        """Collect physical parameters from the user."""
        print("\nI'd like to collect some additional physical parameters to improve my assessment.")
        
        # Body temperature
        while True:
            temp = input("What is your body temperature in °C (or 'skip')? > ")
            if temp.lower() == 'skip':
                break
            try:
                temp = float(temp)
                if 35 <= temp <= 42:  # Reasonable temperature range
                    self.physical_params['temperature'] = temp
                    break
                else:
                    print("Please enter a valid temperature between 35°C and 42°C.")
            except ValueError:
                print("Please enter a valid number or 'skip'.")
        
        # Weight
        while True:
            weight = input("What is your weight in kg (or 'skip')? > ")
            if weight.lower() == 'skip':
                break
            try:
                weight = float(weight)
                if 20 <= weight <= 250:  # Reasonable weight range
                    self.physical_params['weight'] = weight
                    break
                else:
                    print("Please enter a valid weight between 20kg and 250kg.")
            except ValueError:
                print("Please enter a valid number or 'skip'.")
        
        # Height
        while True:
            height = input("What is your height in cm (or 'skip')? > ")
            if height.lower() == 'skip':
                break
            try:
                height = float(height)
                if 50 <= height <= 250:  # Reasonable height range
                    self.physical_params['height'] = height
                    break
                else:
                    print("Please enter a valid height between 50cm and 250cm.")
            except ValueError:
                print("Please enter a valid number or 'skip'.")
        
        # Calculate BMI if both weight and height are available
        if 'weight' in self.physical_params and 'height' in self.physical_params:
            height_m = self.physical_params['height'] / 100  # Convert cm to m
            bmi = self.physical_params['weight'] / (height_m * height_m)
            self.physical_params['bmi'] = round(bmi, 1)
        
        # Blood pressure (optional)
        bp = input("What is your blood pressure (e.g., '120/80' or 'skip')? > ")
        if bp.lower() != 'skip':
            try:
                if '/' in bp:
                    systolic, diastolic = map(int, bp.split('/'))
                    self.physical_params['blood_pressure'] = f"{systolic}/{diastolic}"
            except:
                pass  # Ignore invalid blood pressure
    
    def get_disease_category(self, disease):
        """Get the category of a disease."""
        for category, diseases in self.disease_categories.items():
            if disease in diseases:
                return category
        return 'general'  # Default category
    
    def get_dietary_recommendations(self, disease):
        """Get dietary recommendations for a disease."""
        category = self.get_disease_category(disease)
        return self.diet_recommendations.get(category, self.diet_recommendations['general'])
    
    def format_diagnosis(self, disease, confidence):
        """Format the diagnosis result with physical parameters and dietary recommendations."""
        if not disease:
            return "\n===== Medical Assessment =====\n\nBased on the information provided, I cannot make a specific diagnosis. The symptoms you've described are too general or may not match any specific condition in our database. Please consult a healthcare professional for a proper evaluation."
        
        # Get disease details
        description = ""
        description_row = self.descriptions[self.descriptions['Disease'] == disease]
        if not description_row.empty:
            description = description_row['Description'].iloc[0]
        
        # Get precautions
        precautions = []
        precaution_row = self.precautions[self.precautions['Disease'] == disease]
        if not precaution_row.empty:
            for i in range(1, 5):
                col = f'Precaution_{i}'
                if col in self.precautions.columns and pd.notna(precaution_row[col].iloc[0]):
                    precautions.append(precaution_row[col].iloc[0])
        
        # Format the diagnosis
        result = f"\n===== Medical Assessment =====\n\n"
        
        # Doctor-like introduction
        if confidence > 0.7:
            result += f"Based on your symptoms, you most likely have {disease}.\n"
        elif confidence > 0.5:
            result += f"Your symptoms suggest {disease}, though further evaluation may be needed.\n"
        else:
            result += f"Your symptoms are consistent with {disease}, but this is a preliminary assessment only.\n"
        
        result += f"Diagnostic confidence: {confidence:.2f}\n\n"
        
        # Physical parameters section
        if self.physical_params:
            result += "Physical parameters:\n"
            if 'temperature' in self.physical_params:
                temp = self.physical_params['temperature']
                temp_status = "elevated" if temp > 37.5 else "normal" if temp >= 36.5 else "below normal"
                result += f"- Body temperature: {temp}°C ({temp_status})\n"
            
            if 'weight' in self.physical_params:
                result += f"- Weight: {self.physical_params['weight']} kg\n"
            
            if 'height' in self.physical_params:
                result += f"- Height: {self.physical_params['height']} cm\n"
            
            if 'bmi' in self.physical_params:
                bmi = self.physical_params['bmi']
                bmi_category = "underweight" if bmi < 18.5 else "normal" if bmi < 25 else "overweight" if bmi < 30 else "obese"
                result += f"- BMI: {bmi} ({bmi_category})\n"
            
            if 'blood_pressure' in self.physical_params:
                bp = self.physical_params['blood_pressure']
                systolic, diastolic = map(int, bp.split('/'))
                bp_status = "elevated" if systolic > 130 or diastolic > 80 else "normal"
                result += f"- Blood pressure: {bp} mmHg ({bp_status})\n"
            
            result += "\n"
        
        if description:
            result += f"About this condition:\n{description}\n\n"
        
        # Symptom analysis
        result += "Symptom analysis:\n"
        for symptom in self.confirmed_symptoms:
            severity = self.get_symptom_severity(symptom)
            severity_text = "mild" if severity <= 3 else "moderate" if severity <= 6 else "severe"
            result += f"- {symptom.replace('_', ' ')} ({severity_text})\n"
        
        # Dietary recommendations
        result += "\nDietary recommendations:\n"
        diet_recs = self.get_dietary_recommendations(disease)
        for i, rec in enumerate(diet_recs, 1):
            result += f"{i}. {rec}\n"
        
        # Medical advice
        result += "\nMedical recommendations:\n"
        if precautions:
            for i, precaution in enumerate(precautions, 1):
                result += f"{i}. {precaution}\n"
        else:
            result += "1. Rest and stay hydrated\n"
            result += "2. Monitor your symptoms\n"
            result += "3. Seek medical attention if symptoms worsen\n"
        
        result += "\nIMPORTANT: This is an automated assessment and not a medical diagnosis. Please consult a healthcare professional for proper evaluation and treatment."
        
        return result
    
    def run_diagnosis(self):
        """Run the enhanced diagnosis process."""
        # Reset symptoms and parameters
        self.confirmed_symptoms = set()
        self.denied_symptoms = set()
        self.physical_params = {}
        
        # Get initial input
        print("\nPlease describe your symptoms in detail:")
        initial_input = input("> ")
        
        # Extract initial symptoms
        initial_symptoms = self.extract_symptoms_from_text(initial_input)
        self.confirmed_symptoms.update(initial_symptoms)
        
        if initial_symptoms:
            print(f"\nI've noted these symptoms: {', '.join(s.replace('_', ' ') for s in initial_symptoms)}")
            print("Let me ask some additional questions to refine my assessment.")
        else:
            print("\nI need more specific information about your symptoms. Let me ask some questions.")
        
        # Ask exactly 8 questions
        questions_asked = 0
        
        while questions_asked < self.num_questions:
            # Get the next question
            next_symptom = self.get_next_question()
            
            if not next_symptom:
                break  # No more relevant questions to ask
            
            # Ask the question
            question = self.format_question(next_symptom, questions_asked + 1)
            print(f"\n{question}")
            
            # Get response with immediate processing
            response = input("> ")
            
            # Process response immediately
            if response.lower().startswith(('y', 'yes')):
                self.confirmed_symptoms.add(next_symptom)
            elif response.lower().startswith(('n', 'no')):
                self.denied_symptoms.add(next_symptom)
            # For other responses, we don't add to either set
            
            questions_asked += 1
        
        # Collect physical parameters
        self.collect_physical_parameters()
        
        # Make final prediction
        disease, confidence = self.predict_disease()
        
        # Return diagnosis with dietary recommendations
        return self.format_diagnosis(disease, confidence)

def main():
    """Main function to run the enhanced diagnosis system."""
    # Initialize the system
    system = EnhancedDiagnosisSystem()
    
    print("\n===== Enhanced Medical Assessment System =====")
    print("I'll ask 8 targeted questions and collect physical parameters to assess your condition.")
    print("Type 'quit' at any time to exit.")
    
    while True:
        try:
            # Run the diagnosis
            diagnosis = system.run_diagnosis()
            print(diagnosis)
            
            # Ask if the user wants to start a new diagnosis
            print("\nWould you like another assessment? (yes/no)")
            response = input("> ")
            
            if response.lower() == 'quit' or not response.lower().startswith(('y', 'yes')):
                print("Thank you for using the Enhanced Medical Assessment System. Take care of your health!")
                break
                
        except KeyboardInterrupt:
            print("\nAssessment interrupted.")
            break
        except Exception as e:
            print(f"\nAn error occurred: {e}")
            break

if __name__ == "__main__":
    main()


Loading comprehensive medical data...
Medical data loaded successfully!

===== Enhanced Medical Assessment System =====
I'll ask 8 targeted questions and collect physical parameters to assess your condition.
Type 'quit' at any time to exit.

Please describe your symptoms in detail:

I need more specific information about your symptoms. Let me ask some questions.

Question 1/8: Have you noticed high fever?

Question 2/8: I'd like to know if you have high fever?

Question 3/8: Have you been experiencing high fever?

Question 4/8: To narrow down the diagnosis, are you experiencing high fever?

Question 5/8: I'd like to know if you have high fever?

Question 6/8: Are you suffering from high fever?

Question 7/8: This is important: have you had any high fever?

Question 8/8: Do you have any high fever?

I'd like to collect some additional physical parameters to improve my assessment.
Please enter a valid number or 'skip'.
Please enter a valid number or 'skip'.
Please enter a valid number or

In [ ]:
# Install required packages
import pandas as pd
import numpy as np
import string
import re
import os
import pickle
import warnings
warnings.filterwarnings('ignore')

# For text processing
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# For neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# For model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

class NeuralHealthPredictor:
    """A neural network-based health diagnosis predictor with NLP capabilities."""
    
    def __init__(self, model_path='health_nn_model.h5', vectorizer_path='vectorizer.pkl', 
                 label_encoder_path='label_encoder.pkl', train_model=False):
        """Initialize the neural health predictor."""
        print("Initializing Neural Health Predictor...")
        
        # Load the dataset
        self.dataset = pd.read_csv('./Datasets/dataset.csv')
        self.descriptions = pd.read_csv('./Datasets/symptom_Description.csv')
        self.precautions = pd.read_csv('./Datasets/symptom_precaution.csv')
        
        # Extract all symptoms
        self.all_symptoms = self.extract_all_symptoms()
        
        # Create disease-symptom dictionary
        self.disease_symptom_dict = self.create_disease_symptom_dict()
        
        # Initialize NLP components
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Model paths
        self.model_path = model_path
        self.vectorizer_path = vectorizer_path
        self.label_encoder_path = label_encoder_path
        
        # Load or train the model
        if train_model or not os.path.exists(model_path):
            print("Training new neural network model...")
            self.prepare_training_data()
            self.train_model()
        else:
            print("Loading pre-trained model...")
            self.load_model()
        
        print("Neural Health Predictor initialized successfully!")
    
    def extract_all_symptoms(self):
        """Extract all unique symptoms from the dataset."""
        symptom_cols = [col for col in self.dataset.columns if 'Symptom_' in col]
        all_symptoms = set()
        
        for col in symptom_cols:
            symptoms = self.dataset[col].dropna().unique()
            all_symptoms.update([s.strip() for s in symptoms if s.strip()])
        
        return sorted(list(all_symptoms))
    
    def create_disease_symptom_dict(self):
        """Create a dictionary mapping diseases to their symptoms."""
        disease_symptom_dict = {}
        symptom_cols = [col for col in self.dataset.columns if 'Symptom_' in col]
        
        for disease in self.dataset['Disease'].unique():
            disease_rows = self.dataset[self.dataset['Disease'] == disease]
            symptoms = set()
            
            for _, row in disease_rows.iterrows():
                for col in symptom_cols:
                    if pd.notna(row[col]) and row[col] != '':
                        symptoms.add(row[col].strip())
            
            disease_symptom_dict[disease] = sorted(list(symptoms))
        
        return disease_symptom_dict
    
    def preprocess_text(self, text):
        """Preprocess text for NLP."""
        # Convert to lowercase and remove punctuation
        text = text.lower().translate(str.maketrans('', '', string.punctuation))
        
        # Tokenize
        tokens = text.split()
        
        # Remove stop words and lemmatize
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens if word not in self.stop_words]
        
        return ' '.join(tokens)
    
    def prepare_training_data(self):
        """Prepare training data for the neural network."""
        # Create text descriptions of symptoms for each disease
        texts = []
        labels = []
        
        for disease, symptoms in self.disease_symptom_dict.items():
            # Create multiple text variations for each disease
            for _ in range(5):  # Create 5 variations per disease
                # Randomly select 70-100% of symptoms
                num_symptoms = max(1, int(np.random.uniform(0.7, 1.0) * len(symptoms)))
                selected_symptoms = np.random.choice(symptoms, num_symptoms, replace=False)
                
                # Create a text description
                text = ' '.join([s.replace('_', ' ').lower() for s in selected_symptoms])
                
                texts.append(text)
                labels.append(disease)
        
        # Create a DataFrame
        self.train_df = pd.DataFrame({'text': texts, 'disease': labels})
        
        # Preprocess texts
        self.train_df['processed_text'] = self.train_df['text'].apply(self.preprocess_text)
        
        # Create vectorizer
        self.vectorizer = CountVectorizer(max_features=1000)
        X = self.vectorizer.fit_transform(self.train_df['processed_text']).toarray()
        
        # Encode labels
        self.diseases = sorted(list(self.train_df['disease'].unique()))
        self.label_to_idx = {disease: i for i, disease in enumerate(self.diseases)}
        self.idx_to_label = {i: disease for disease, i in self.label_to_idx.items()}
        
        y = np.array([self.label_to_idx[disease] for disease in self.train_df['disease']])
        y = to_categorical(y, num_classes=len(self.diseases))
        
        # Split data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Save vectorizer and label encoder
        with open(self.vectorizer_path, 'wb') as f:
            pickle.dump(self.vectorizer, f)
        
        with open(self.label_encoder_path, 'wb') as f:
            pickle.dump({'label_to_idx': self.label_to_idx, 'idx_to_label': self.idx_to_label}, f)
    
    def build_model(self):
        """Build the neural network model."""
        input_dim = self.X_train.shape[1]
        output_dim = len(self.diseases)
        
        model = Sequential([
            Input(shape=(input_dim,)),
            Dense(512, activation='relu'),
            Dropout(0.3),
            Dense(256, activation='relu'),
            Dropout(0.3),
            Dense(128, activation='relu'),
            Dropout(0.3),
            Dense(output_dim, activation='softmax')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def train_model(self):
        """Train the neural network model."""
        self.model = self.build_model()
        
        # Callbacks
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
        
        model_checkpoint = ModelCheckpoint(
            self.model_path,
            monitor='val_loss',
            save_best_only=True
        )
        
        # Train the model
        history = self.model.fit(
            self.X_train, self.y_train,
            epochs=50,
            batch_size=32,
            validation_data=(self.X_test, self.y_test),
            callbacks=[early_stopping, model_checkpoint],
            verbose=1
        )
        
        # Evaluate the model
        y_pred_proba = self.model.predict(self.X_test)
        y_pred = np.argmax(y_pred_proba, axis=1)
        y_true = np.argmax(self.y_test, axis=1)
        
        accuracy = accuracy_score(y_true, y_pred)
        print(f"Model accuracy: {accuracy:.4f}")
        
        # Save the model
        self.model.save(self.model_path)
    
    def load_model(self):
        """Load the pre-trained model and components."""
        try:
            self.model = load_model(self.model_path)
            
            with open(self.vectorizer_path, 'rb') as f:
                self.vectorizer = pickle.load(f)
            
            with open(self.label_encoder_path, 'rb') as f:
                encoders = pickle.load(f)
                self.label_to_idx = encoders['label_to_idx']
                self.idx_to_label = encoders['idx_to_label']
                self.diseases = sorted(list(self.label_to_idx.keys()))
            
            print("Model and components loaded successfully.")
        except Exception as e:
            print(f"Error loading model: {e}")
            print("Training new model instead...")
            self.prepare_training_data()
            self.train_model()
    
    def predict_from_text(self, text, top_n=3):
        """Predict diseases from text description using the neural network."""
        # Preprocess the text
        processed_text = self.preprocess_text(text)
        
        # Vectorize
        X = self.vectorizer.transform([processed_text]).toarray()
        
        # Predict
        y_pred_proba = self.model.predict(X)[0]
        
        # Get top N predictions
        top_indices = np.argsort(y_pred_proba)[-top_n:][::-1]
        
        results = []
        for idx in top_indices:
            disease = self.idx_to_label[idx]
            confidence = y_pred_proba[idx]
            
            # Get disease details
            description = ""
            description_row = self.descriptions[self.descriptions['Disease'] == disease]
            if not description_row.empty:
                description = description_row['Description'].iloc[0]
            
            # Get precautions
            precautions = []
            precaution_row = self.precautions[self.precautions['Disease'] == disease]
            if not precaution_row.empty:
                for i in range(1, 5):
                    col = f'Precaution_{i}'
                    if col in self.precautions.columns and pd.notna(precaution_row[col].iloc[0]):
                        precautions.append(precaution_row[col].iloc[0])
            
            # Extract symptoms from text that match this disease
            disease_symptoms = set(self.disease_symptom_dict[disease])
            matched_symptoms = []
            
            for symptom in disease_symptoms:
                clean_symptom = symptom.replace('_', ' ').lower()
                if clean_symptom in text.lower():
                    matched_symptoms.append(symptom)
            
            results.append({
                'disease': disease,
                'confidence': float(confidence),
                'description': description,
                'precautions': precautions,
                'matched_symptoms': matched_symptoms,
                'all_symptoms': list(disease_symptoms)
            })
        
        return results
    
    def extract_symptoms_from_text(self, text):
        """Extract symptoms from natural language text."""
        # Convert to lowercase and remove punctuation
        text = text.lower().translate(str.maketrans('', '', string.punctuation))
        
        # Simple keyword matching for symptoms
        extracted_symptoms = []
        for symptom in self.all_symptoms:
            clean_symptom = symptom.replace('_', ' ').lower()
            if clean_symptom in text:
                extracted_symptoms.append(symptom)
        
        return extracted_symptoms
    
    def format_prediction(self, text, predictions):
        """Format the prediction results for display."""
        result = f"\n===== Neural Network Health Assessment =====\n\n"
        result += f"Your input: \"{text}\"\n\n"
        
        # Extract symptoms
        extracted_symptoms = self.extract_symptoms_from_text(text)
        if extracted_symptoms:
            result += f"Identified symptoms: {', '.join(s.replace('_', ' ') for s in extracted_symptoms)}\n\n"
        else:
            result += "No specific symptoms identified from your input.\n\n"
        
        # Top prediction
        top_prediction = predictions[0]
        disease = top_prediction['disease']
        confidence = top_prediction['confidence']
        
        result += f"Top prediction: {disease}\n"
        # result += f"Confidence: {confidence:.2f}\n\n"
        
        if top_prediction['description']:
            result += f"Description:\n{top_prediction['description']}\n\n"
        
        if top_prediction['matched_symptoms']:
            result += "Matched symptoms:\n"
            for symptom in top_prediction['matched_symptoms']:
                result += f"- {symptom.replace('_', ' ')}\n"
        
        other_symptoms = [s for s in top_prediction['all_symptoms'] if s not in top_prediction['matched_symptoms']]
        if other_symptoms:
            result += "\nOther common symptoms:\n"
            for symptom in other_symptoms[:5]:  # Show only top 5
                result += f"- {symptom.replace('_', ' ')}\n"
        
        if top_prediction['precautions']:
            result += "\nRecommended precautions:\n"
            for i, precaution in enumerate(top_prediction['precautions'], 1):
                result += f"{i}. {precaution}\n"
        
        # Other possible conditions
        if len(predictions) > 1:
            result += "\nOther possible conditions:\n"
            for pred in predictions[1:]:
                result += f"- {pred['disease']}\n"
        
        result += "\nNote: This is not a medical diagnosis. Please consult a healthcare professional."
        
        return result
    
    def diagnose(self, text):
        """Process user input and return a diagnosis using the neural network."""
        predictions = self.predict_from_text(text)
        return self.format_prediction(text, predictions)

def main():
    """Main function to run the neural health predictor."""
    # Check if model exists
    model_exists = os.path.exists('health_nn_model.h5')
    
    # Initialize the predictor
    predictor = NeuralHealthPredictor(train_model=not model_exists)
    
    print("\n===== Neural Network Health Diagnosis System =====")
    print("Describe your symptoms and I'll predict possible conditions.")
    print("Type 'quit' to exit.")
    
    while True:
        # Get user input
        print("\nPlease describe your symptoms:")
        user_input = input("> ")
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("Thank you for using the Neural Network Health Diagnosis System. Goodbye!")
            break
        
        # Process the input and display the diagnosis
        diagnosis = predictor.diagnose(user_input)
        print(diagnosis)

if __name__ == "__main__":
    # Try to import nltk data
    try:
        import nltk
        nltk.download('wordnet', quiet=True)
        nltk.download('stopwords', quiet=True)
    except:
        print("Warning: NLTK data download failed. Some NLP features may not work properly.")
    
    main()


Initializing Neural Health Predictor...


Loading pre-trained model...
Model and components loaded successfully.
Neural Health Predictor initialized successfully!

===== Neural Network Health Diagnosis System =====
Describe your symptoms and I'll predict possible conditions.
Type 'quit' to exit.

Please describe your symptoms:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step

===== Neural Network Health Assessment =====

Your input: "chest and knee pain"

Identified symptoms: knee pain

Top prediction: Hypertension 

Other common symptoms:
- dizziness
- headache
- chest pain
- lack of concentration
- loss of balance

Recommended precautions:
1. meditation
2. salt baths
3. reduce stress
4. get proper sleep

Other possible conditions:
- GERD
- Osteoarthristis

Note: This is not a medical diagnosis. Please consult a healthcare professional.

Please describe your symptoms:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

===== Neural Network Health Assessment =====

Your input: "chest pain and knee pain"

Identified symptoms: chest pain, knee pain

T